# 02_plot_headmotion_figure

In [1]:
import numpy as np
from os.path import join as pjoin
from os.path import isdir
import os
import matplotlib.pyplot as plt
from matplotlib import cm, colors
import mne_bids
import mne
from mne_bids import write_raw_bids, BIDSPath
from scipy import stats
import re
from scipy import signal
import pandas as pd
from scipy import signal, fftpack

In [2]:
########################################################
# what is the difference between this and the last cell? 
########################################################

def plot_sub_hmv_hist(hpi, fig_info, save_pth=None):
    """
    use hpi data to plot histgram of head movement
    
    Parameters
    ----------
        hpi: list of hpi channels data, include channel name and data
        fig_info: [sub_idx, run_idx]
        save_pth: str
    """
    
    coord = ['nasion', 'lpa', 'rpa']
    color = ['lightskyblue', 'lightgreen', 'pink']
    hpi_de = []
    for chn, val in hpi:
        hpi_de.append((chn, 1000*(val-val[0])))

    fig, axes = plt.subplots(1,3,figsize=(16,5))
    fig.suptitle('Head Movement')
    for j in np.arange(3):
        mv = [np.sqrt(hpi_de[j*4][1][n]**2 + hpi_de[j*4+1][1][n]**2 + hpi_de[j*4+2][1][n]**2) for n in np.arange(len(hpi_de[j-1][1]))]
        x, bins, p = axes[j].hist(mv, alpha=0.4, density=True, bins=20, label=coord[j], color=color[j])
        height = []
        for item in p:
            height.append(item.get_height()/sum(x))
            item.set_height(item.get_height()/sum(x))
        axes[j].set_ylim(top=np.ceil(max(height)*10)/10)
        axes[j].legend()
        axes[j].set_xlabel('mm')
        axes[j].set_ylabel('normalized histgram')
        
    if save_pth is not None:
        plt.savefig(pjoin(save_pth, f'sub-{fig_info[0]}_run-{fig_info[1]}_deriv.jpeg'),dpi=600)
        plt.close()

In [3]:
# define variables
sub_list = ['{0:0>2d}'.format(sub) for sub in np.arange(1,12)]
run_list = ['{0:0>2d}'.format(run) for run in np.arange(1,9)]
coord = ['nasion', 'lpa', 'rpa']

# set path
bids_root = '/nfs/e5/studyforrest/forrest_movie_meg/meg_preproc_data_bak/'
results_pth = pjoin(os.getcwd(),'tech_val_results')
fig_save_pth = pjoin(os.getcwd(),'tech_val_results', 'head_mv_fig')

if os.path.exists(fig_save_pth) is False: os.mkdir(fig_save_pth)

In [4]:
# load data
hpi_data = pd.read_pickle(pjoin(results_pth, 'hpi_data.pickle'))

In [ ]:
# plot figure
head_mv = []
coord = ['nasion', 'lpa', 'rpa']
for j, coil in enumerate(coord):
    coil_mv = np.array([])
    for sub in sub_list:
        for run in run_list:
            hpi = hpi_data[sub][run]
            hpi_de = []
            for i, (chn, val) in enumerate(hpi):
                hpi_de.append((chn, 1000*(val-val[0])))
            mv = [np.sqrt(hpi_de[j*4][1][n]**2 + hpi_de[j*4+1][1][n]**2 + hpi_de[j*4+2][1][n]**2) for n in np.arange(len(hpi_de[j-1][1]))]
            if np.where(np.array(mv)>10)[0].size > 0:
                print(f'lager head movement: sub-{sub} run-{run} coil:{coil}')
            coil_mv = np.append(coil_mv, np.array(mv))
    head_mv.append((coil, coil_mv))

color = ['lightskyblue', 'lightgreen', 'pink']
fig, axes = plt.subplots(1,3,figsize=(16,5))
fig.suptitle('Head Movement')
for j in np.arange(3):
    x, bins, p = axes[j].hist(mv, alpha=0.4, density=True, bins=50, label=coord[j], color=color[j])
    height = []
    for item in p:
        height.append(item.get_height()/sum(x))
        item.set_height(item.get_height()/sum(x))
    axes[j].set_ylim(top=np.ceil(max(height)*10)/10)
    axes[j].legend()
    axes[j].set_xlabel('mm')
    axes[j].set_ylabel('normalized histgram')

# save fig
plt.savefig(pjoin(save_pth, f'sub-total_run-total_deriv.jpeg'),dpi=600)